# 🤖 LangChain HR Chatbot Notebook
This notebook demonstrates how to build a domain-specific chatbot using LangChain and OpenAI.

In [ ]:
import os
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


## 📥 Load Dataset

In [ ]:
df = pd.read_csv("data/hr_faq.csv")
df.head()


## 🧹 Text Preprocessing and Vector Store Creation

In [ ]:
docs = [f"Q: {row['Question']}\nA: {row['Answer']}" for _, row in df.iterrows()]
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = splitter.create_documents(docs)

embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(texts, embeddings)
vectordb.save_local("model")


## 🧠 Load Model and Create Chat Chain

In [ ]:
vectordb = FAISS.load_local("model", OpenAIEmbeddings())
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb.as_retriever())


## 💬 Ask Questions

In [ ]:
query = "How many leaves do employees get annually?"
response = qa_chain.run(query)
print("🤖 Answer:", response)
